# Cyber Guardian

## Description

Cyber Guardian is a comprehensive application developed by Weiqi Zhang, designed to secure documents through cryptographic operations. The application offers encryption, decryption, signing, and verification functionalities, ensuring data integrity and confidentiality. It also provides a user-friendly interface for managing users and interacting with documents securely.


## Setup and Imports

This section imports all necessary libraries required for the application to function, including handling encryption, storing user data, and creating GUI elements.


In [1]:
import os
import json
import bcrypt
import keyring
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output
import base64
from nacl.signing import SigningKey, VerifyKey
from nacl.secret import SecretBox
from nacl.encoding import HexEncoder, Base64Encoder

# Admin credentials
ADMIN_USERNAME = ''
ADMIN_PASSWORD_HASH = bcrypt.hashpw(''.encode(), bcrypt.gensalt())

# File path for storing user credentials
file_path = 'Cyber_Guard_File.json'

# Generate a new signing key
signing_key = SigningKey.generate()
verify_key = signing_key.verify_key

# Encode the verify key to store it
encoded_verify_key = verify_key.encode(encoder=HexEncoder)

# Save the encoded verify key in the keyring
keyring.set_password("CyberGuardian", "public_signing_key", encoded_verify_key.decode())


## Helper Functions

These functions load and save user data securely, handle password encryption and verification, and manage local storage.


In [2]:
def load_users():
    try:
        with open(file_path, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        print("User data file not found. Creating a new one.")
        return {}
    except json.JSONDecodeError:
        print("Error decoding JSON. Check file format.")
        return {}

def save_users(users):
    with open(file_path, 'w') as file:
        json.dump(users, file, indent=4)
    print("Users saved successfully.")

def hash_password(password):
    return bcrypt.hashpw(password.encode(), bcrypt.gensalt()).decode()

def check_password(hashed_password, user_password):
    return bcrypt.checkpw(user_password.encode(), hashed_password.encode())


In [3]:
def test_sign_and_verify(document_path):
    # Generate a new signing key for testing
    signing_key = SigningKey.generate()
    verify_key = signing_key.verify_key

    # Store and retrieve keys (simulate key management)
    keyring.set_password("test", "signing_key", signing_key.encode(encoder=HexEncoder).decode())
    keyring.set_password("test", "verify_key", verify_key.encode(encoder=HexEncoder).decode())

    # Check if the document path exists
    if not os.path.exists(document_path):
        print(f"Document not found: {document_path}")
        return

    # Read the document
    with open(document_path, 'rb') as f:
        document_content = f.read()

    # Sign the document
    signing_key_encoded = keyring.get_password("test", "signing_key").encode()
    signing_key = SigningKey(signing_key_encoded, encoder=HexEncoder)
    signature = signing_key.sign(document_content).signature

    # Verify the signature
    verify_key_encoded = keyring.get_password("test", "verify_key").encode()
    verify_key = VerifyKey(verify_key_encoded, encoder=HexEncoder)
    try:
        verify_key.verify(document_content, signature)
        print("Test verification successful!")
    except Exception as e:
        print("Test verification failed:", e)

# Set the document path relative to the current working directory
current_directory = os.getcwd()
document_file = os.path.join(current_directory, "test.pdf")

test_sign_and_verify(document_file)


Test verification successful!


In [4]:
from nacl.signing import SigningKey
from nacl.encoding import HexEncoder
import keyring

# Generate a new key pair
new_signing_key = SigningKey.generate()
new_verify_key = new_signing_key.verify_key

# Encode and store keys securely
keyring.set_password("CyberGuardian", "signing_key", new_signing_key.encode(encoder=HexEncoder).decode())
keyring.set_password("CyberGuardian", "public_signing_key", new_verify_key.encode(encoder=HexEncoder).decode())

print("New keys generated and stored.")

New keys generated and stored.


In [5]:
stored_signing_key = keyring.get_password("CyberGuardian", "signing_key")
stored_verify_key = keyring.get_password("CyberGuardian", "public_signing_key")

print("Stored Signing Key:", stored_signing_key)
print("Stored Verification Key:", stored_verify_key)


Stored Signing Key: 5a2733f7b0f0a49d4e136b51fea1e50f4ecea0a16b230878200006de5d665eb0
Stored Verification Key: b4c5c69531170b2730dcc6999bc9e01e040c92ef2752fa4defc2bbb44b8f40ad


In [6]:
def test_key_functionality():
    signing_key_encoded = keyring.get_password("CyberGuardian", "signing_key").encode()
    verify_key_encoded = keyring.get_password("CyberGuardian", "public_signing_key").encode()

    signing_key = SigningKey(signing_key_encoded, encoder=HexEncoder)
    verify_key = VerifyKey(verify_key_encoded, encoder=HexEncoder)

    # Test signing and verifying
    message = b"Hello, this is a test message!"
    signature = signing_key.sign(message).signature
    try:
        verify_key.verify(message, signature)
        print("Signature verified successfully!")
    except Exception as e:
        print("Failed to verify signature:", str(e))

test_key_functionality()

Signature verified successfully!


In [7]:
def sign_document(file_path):
    """
    Sign the document specified by file_path and save the signature.
    """
    try:
        # Retrieve the private signing key from secure storage
        signing_key_encoded = keyring.get_password("CyberGuardian", "signing_key")
        if not signing_key_encoded:
            print("Signing key not found in keyring.")
            return
        
        signing_key = SigningKey(signing_key_encoded.encode(), encoder=HexEncoder)

        # Read the document content
        with open(file_path, 'rb') as f:
            document_content = f.read()

        # Sign the document
        signature = signing_key.sign(document_content).signature

        # Save the signature to a file
        signature_file_path = file_path + '.sig'
        with open(signature_file_path, 'wb') as f:
            f.write(signature)
        
        print(f"Document signed successfully. Signature saved as {signature_file_path}")
    except Exception as e:
        print(f"An error occurred during the signing process: {str(e)}")

# Example use
current_directory = os.getcwd()
document_file = os.path.join(current_directory, "test.pdf")

sign_document(document_file)

Document signed successfully. Signature saved as /Users/weiqizhang/DSD_content/Cyber_Guard_Fil/test.pdf.sig


In [8]:
from nacl.signing import VerifyKey
from nacl.encoding import HexEncoder
import keyring
import os

def verify_document(document_path, signature_path):
    """
    Verify the document at document_path using the signature in signature_path.
    """
    try:
        # Retrieve the public key from secure storage
        verify_key_encoded = keyring.get_password("CyberGuardian", "public_signing_key")
        if not verify_key_encoded:
            print("Verification key not found in keyring.")
            return
        
        verify_key = VerifyKey(verify_key_encoded.encode(), encoder=HexEncoder)

        # Read the document content
        with open(document_path, 'rb') as f:
            document_content = f.read()

        # Read the signature
        with open(signature_path, 'rb') as f:
            signature = f.read()

        # Verify the signature
        verify_key.verify(document_content, signature)
        print(f"Verification successful for {document_path}")
    except Exception as e:
        print(f"An error occurred during the verification process: {str(e)}")

# Set paths relative to the current working directory
current_directory = os.getcwd()
document_file = os.path.join(current_directory, "test.pdf")
signature_file = os.path.join(current_directory, "test.pdf.sig")

verify_document(document_file, signature_file)

Verification successful for /Users/weiqizhang/DSD_content/Cyber_Guard_Fil/test.pdf


## User Registration and Login

Handles user authentication and registration, checking credentials against stored data and directing users to appropriate functionalities.


In [9]:
def on_login_clicked(b):
    with output_area:
        clear_output()
        username = username_input.value
        password = password_input.value
        users = load_users()
        if username == ADMIN_USERNAME and bcrypt.checkpw(password.encode(), ADMIN_PASSWORD_HASH):
            print("Administrator login successful")
            show_admin_controls()
        elif username in users and check_password(users[username]['password'], password):
            print("Login successful")
            show_user_controls(username)
        else:
            print("Invalid username or password")

def on_register_clicked(b):
    with output_area:
        clear_output()
        username = username_input.value
        password = password_input.value
        users = load_users()
        if username in users:
            print("Username already exists")
        else:
            users[username] = {
                'password': hash_password(password),
                'created': datetime.now().strftime("%Y-%m-%d"),
                'role': 'user'
            }
            save_users(users)
            print("User registered successfully")
            show_user_controls(username)


## User Interface Components

Defines interactive GUI elements using `ipywidgets`, including input fields, buttons, and output areas.


In [10]:
username_input = widgets.Text(description="Username:")
password_input = widgets.Password(description="Password:")
login_button = widgets.Button(description="Login")
register_button = widgets.Button(description="Register")
output_area = widgets.Output()

login_button.on_click(on_login_clicked)
register_button.on_click(on_register_clicked)

## Admin and User Controls

This section includes functions that display admin-specific controls such as viewing all registered users, and user-specific controls like document management options (encrypt, decrypt, sign, verify). Each set of controls is displayed based on the user's role after a successful login.


In [11]:
def show_admin_controls():
    clear_output()
    admin_output = widgets.Output()
    refresh_button = widgets.Button(description="Refresh User List")
    user_list = widgets.Select(options=[])

    def refresh_users():
        with admin_output:
            clear_output()
            users = load_users()
            user_list.options = [(user, details) for user, details in users.items()]
            display(user_list)

    refresh_button.on_click(lambda b: refresh_users())
    display(refresh_button, admin_output)
    refresh_users()

def show_user_controls(username):
    clear_output()
    user_output = widgets.Output()

    # Widget for file upload
    file_selector = widgets.FileUpload(
        description="Upload Document",
        multiple=False,
        accept='',  # You can restrict file types e.g., '.txt,.pdf'
        style={'description_width': 'initial'}
    )

    # Define the function to handle file upload
    def handle_file_upload(change):
        for uploaded_filename in change['new']:
            file_info = file_selector.value[uploaded_filename]
            with open(uploaded_filename, 'wb') as f:
                f.write(file_info['content'])
            print(f"{uploaded_filename} has been saved to the current directory and is ready for processing.")

    # Observe file upload events
    file_selector.observe(handle_file_upload, names='data')

    # Buttons for document management actions
    encrypt_button = widgets.Button(description="Encrypt Document")
    decrypt_button = widgets.Button(description="Decrypt Document")
    sign_button = widgets.Button(description="Sign Document")
    verify_button = widgets.Button(description="Verify Document")


    # Function to handle document encryption
    def encrypt_document(b):
        with user_output:
            clear_output()
            print("Encrypt button clicked")
            if not file_selector.value:
                print("No file selected. Please upload a file.")
                return
            
            try:
                # Extract the first element from the tuple and access its dictionary
                file_info = file_selector.value[0]  # Assuming the tuple contains at least one item

                secret_key_encoded = keyring.get_password("CyberGuardian", "encryption_key")
                if not secret_key_encoded:
                    print("Encryption key not found in keyring.")
                    return
                
                secret_key = base64.b64decode(secret_key_encoded)
                box = SecretBox(secret_key)
                encrypted = box.encrypt(file_info['content'])
                encrypted_file_path = file_info['name'] + ".enc"
                with open(encrypted_file_path, 'wb') as enc_file:
                    enc_file.write(encrypted)
                print(f"File encrypted successfully. Saved as {encrypted_file_path}")
            except Exception as e:
                print(f"An error occurred during the encryption process: {str(e)}")

    
    def decrypt_document(b):
        with user_output:
            clear_output()
            print("Decrypt button clicked")
            if not file_selector.value:
                print("No file selected. Please upload a file.")
                return

            try:
                file_info = file_selector.value[0]  # Accessing the first file info in the tuple
                content = file_info['content']
                if isinstance(content, memoryview):
                    content = content.tobytes()  # Convert memoryview to bytes

                secret_key_encoded = keyring.get_password("CyberGuardian", "encryption_key")
                if not secret_key_encoded:
                    print("Encryption key not found in keyring.")
                    return

                secret_key = base64.b64decode(secret_key_encoded)
                box = SecretBox(secret_key)
                decrypted = box.decrypt(content)  # Use the converted bytes here
                decrypted_file_path = file_info['name'].replace(".enc", "")
                with open(decrypted_file_path, 'wb') as dec_file:
                    dec_file.write(decrypted)
                print(f"File decrypted successfully. Saved as {decrypted_file_path}")
            except Exception as e:
                print(f"An error occurred during the decryption process: {str(e)}")


    # Similarly define decrypt_document, sign_document, verify_document'
    def sign_document(b):
        with user_output:
            clear_output()
            print("Sign button clicked")
            if not file_selector.value:
                print("No file selected. Please upload a file.")
                return

            try:
                file_info = file_selector.value[0]  # Accessing the first file info in the tuple
                content = file_info['content']
                if isinstance(content, memoryview):
                    content = content.tobytes()  # Convert memoryview to bytes

                signing_key_encoded = keyring.get_password("CyberGuardian", "signing_key")
                if not signing_key_encoded:
                    print("Signing key not found in keyring.")
                    return

                signing_key = SigningKey(signing_key_encoded, encoder=HexEncoder)
                signature = signing_key.sign(content).signature  # Use the converted bytes here
                signature_file_path = file_info['name'] + ".sig"
                with open(signature_file_path, 'wb') as sig_file:
                    sig_file.write(signature)
                print(f"Document signed successfully. Signature saved as {signature_file_path}")
            except Exception as e:
                print(f"An error occurred during the signing process: {str(e)}")

        # Retrieve the verification key from the keyring
    verify_key_encoded = keyring.get_password("CyberGuardian", "public_signing_key")
    if not verify_key_encoded:
        print("Verification key not found in keyring.")
        return
    else:
        verify_key = VerifyKey(verify_key_encoded.encode(), encoder=HexEncoder)



    def verify_document(b):
        with user_output:
            clear_output()
            print("Verify button clicked")
        
            if not file_selector.value:
                print("No file selected. Please upload a signature file.")
                return
        
            # Retrieve the verification key from the keyring
            verify_key_encoded = keyring.get_password("CyberGuardian", "public_signing_key")
            if not verify_key_encoded:
                print("Verification key not found in keyring.")
                return
            verify_key = VerifyKey(verify_key_encoded.encode(), encoder=HexEncoder)
        
            try:
                file_info = file_selector.value[0]
                signature_filename = file_info['name']
        
                if not signature_filename.endswith('.sig'):
                    print("The uploaded file does not appear to be a signature file.")
                    return
                    
                document_filename = signature_filename[:-4]  # Assuming '.sig' is at the end
                document_path = os.path.join(os.getcwd(), document_filename)
        
                if os.path.exists(document_path):
                    with open(document_path, 'rb') as doc_file:
                        document_content = doc_file.read()
        
                    signature_content = file_info['content']
                    if isinstance(signature_content, memoryview):
                        signature_content = signature_content.tobytes()
        
                    verify_key.verify(document_content, signature_content)
                    print(f"Verification successful for {document_filename}")
                else:
                    print(f"No corresponding document found for {signature_filename} in the current directory.")
            except Exception as e:
                print(f"An error occurred during the verification process: {str(e)}")


     
    # Binding buttons to their respective functions
    encrypt_button.on_click(encrypt_document)
    decrypt_button.on_click(decrypt_document)
    sign_button.on_click(sign_document)
    verify_button.on_click(verify_document)

    # Display the widgets
    display(widgets.VBox([file_selector, encrypt_button, decrypt_button, sign_button, verify_button, user_output]))

display(username_input, password_input, login_button, register_button, output_area)

Text(value='', description='Username:')

Password(description='Password:')

Button(description='Login', style=ButtonStyle())

Button(description='Register', style=ButtonStyle())

Output()

## Conclusion

Cyber Guardian provides an intuitive interface for managing and securing documents through cryptographic operations. The comprehensive functionality includes secure user management, robust encryption, decryption, and signing, making it a powerful solution for data protection.